In [174]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor


from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

data = r"C:\Users\hp\Documents\m3\ih-datamadpt-0124-project-m-3\diamonds_train.db"

#Conectar a la base de datos

conn = sqlite3.connect(data)

table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:", table)

#verificar si hay tablas disponibles

if table.empty:
    print("No table found in the database.")
else:
 
    table_name = table['name'][0]
    

# Leer datos de la tabla seleccionada

    df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

    
  




Tables in the database:                      name
0     diamonds_dimensions
1  diamonds_transactional
2     diamonds_properties
3            diamonds_cut
4          diamonds_color
5        diamonds_clarity
6           diamonds_city


In [175]:
dimensions = pd.read_sql_query("SELECT * FROM diamonds_dimensions;", conn)
transactional = pd.read_sql_query("SELECT * FROM diamonds_transactional;", conn)
properties = pd.read_sql_query("SELECT * FROM diamonds_properties;", conn)
cut = pd.read_sql_query("SELECT * FROM diamonds_cut;", conn)
color = pd.read_sql_query("SELECT * FROM diamonds_color;", conn)
clarity = pd.read_sql_query("SELECT * FROM diamonds_clarity;", conn)
city = pd.read_sql_query("SELECT * FROM diamonds_city;", conn)

In [176]:
diamonds= pd.merge(city, transactional)

diamonds2= pd.merge( properties, cut)
diamonds3= pd.merge( diamonds2, color)
diamonds4= pd.merge(diamonds3, clarity)

In [177]:
columns_to_drop= ["cut_id", "clarity_id", "color_id"]
diamonds4.drop(columns=columns_to_drop, axis=1, inplace=True)


In [178]:
diamonds5= pd.merge(diamonds4, dimensions)




In [179]:
diamonds6=pd.merge(diamonds5,diamonds)

columns_to_drop= ["index_id", "city_id", "city"]
diamonds6.drop(columns=columns_to_drop, axis=1, inplace=True)



In [180]:
cut_color_clarity = diamonds6[['cut','clarity']]
diamonds6 = diamonds6.drop(columns=['cut','clarity'])
diamonds6 = pd.concat([diamonds6, cut_color_clarity], axis=1)

diamonds6

,color,depth,table,x,y,z,price,carat,cut,clarity
0,J,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,VS2
1,J,60.5,56.0,7.19,7.12,4.33,6254,1.35,Premium,VS2
2,J,58.4,59.0,7.40,7.27,4.28,6157,1.36,Premium,VS2
3,J,61.3,60.0,4.65,4.68,2.86,581,0.39,Premium,VS2
4,J,58.6,58.0,6.85,6.81,4.00,4422,1.12,Premium,VS2
...,...,...,...,...,...,...,...,...,...,...
40450,F,62.0,58.0,4.25,4.30,2.65,886,0.30,Ideal,IF
40451,F,61.4,55.0,6.61,6.68,4.09,11846,1.09,Ideal,IF
40452,F,61.7,56.0,4.79,4.83,2.97,1439,0.41,Ideal,IF
40453,F,63.0,55.0,4.13,4.16,2.61,760,0.27,Ideal,IF


In [181]:
#encoding

cat_cols = ['cut','clarity',"color"]
diamonds6_encoded = diamonds6[cat_cols]



In [182]:
cat_list = []
for col in cat_cols:
    cat = diamonds6[col].unique()
    cat_num = len(cat)
    cat_dict = {"categorical_variable":col,
                "number_of_possible_values":cat_num,
                "values":cat}
    cat_list.append(cat_dict)
    
cat_df = pd.DataFrame(cat_list).sort_values(by="number_of_possible_values",
                                                ascending=False).reset_index(drop=True)



In [183]:
#encoding

cat_cols = ["clarity",'cut',"color"]
diamonds_one_hot_encoding = pd.get_dummies(diamonds6, columns=cat_cols, drop_first=True)

# Convertir las columnas booleanas a enteros
diamonds_one_hot_encoding = diamonds_one_hot_encoding.astype(int)

diamonds_one_hot_encoding





,depth,table,x,y,z,price,carat,clarity_IF,clarity_SI1,clarity_SI2,...,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J
0,62,58,6,6,4,4268,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,60,56,7,7,4,6254,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,58,59,7,7,4,6157,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,61,60,4,4,2,581,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,58,58,6,6,4,4422,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,62,58,4,4,2,886,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
40451,61,55,6,6,4,11846,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
40452,61,56,4,4,2,1439,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
40453,63,55,4,4,2,760,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0


In [184]:
X = diamonds_one_hot_encoding.drop(columns="price")
y = diamonds_one_hot_encoding ["price"]


In [190]:

# Scaling


#scaler = StandardScaler()

#scaler = MinMaxScaler()

scaler = RobustScaler()
scaled_data = scaler.fit_transform(X)
scaled_data

scaled_df = pd.DataFrame(scaled_data)
scaled_df.describe()



,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,...,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,0.300334,0.145783,0.124682,0.125127,0.003560,0.394957,0.032802,0.240984,0.171277,0.149944,...,0.090545,0.400939,0.253615,0.224818,0.181065,0.177407,0.208998,0.155160,0.099666,0.052898
std,1.454259,0.746381,0.569796,0.580551,0.415292,0.569129,0.178120,0.427686,0.376755,0.357021,...,0.286965,0.490095,0.435085,0.417467,0.385077,0.382017,0.406598,0.362062,0.299558,0.223833
min,-18.000000,-4.666667,-2.500000,-2.500000,-1.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-0.333333,-0.500000,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.666667,0.500000,0.500000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,18.000000,12.666667,2.500000,26.500000,2.500000,4.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

X_train: (32364, 23), X_test: (8091, 23), y_train: (32364,), y_test: (8091,)


In [192]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

print(type(model))

<class 'sklearn.ensemble._forest.RandomForestRegressor'>


In [193]:
model.fit(X_train, y_train)

print('Model:', model, '\n')

Model: RandomForestRegressor() 



In [194]:
predictions = model.predict(X_test)

print(type(predictions))

<class 'numpy.ndarray'>


In [195]:
rmse = mean_squared_error(y_test, predictions, squared=False)
rmse

929.6512579233107

In [196]:
filename = r"C:\Users\hp\Documents\m3\ih-datamadpt-0124-project-m-3\Modelos/test_1.sav"
pickle.dump(model, open(filename, 'wb'))
print('Your model has been saved with  pickle!!!')

Your model has been saved with  pickle!!!
